In [5]:
import pandas as pd
import lxml
import html5lib
import bs4 as soup
import requests
import json
import time
!pip install folium
import folium
import requests
import regex as re
import string
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
from folium.plugins import MarkerCluster
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

from pandas.io.json import json_normalize

# packages successful if no error in console
from jsonschema._validators import items
print("All packages installed")

     |████████████████████████████████| 94 kB 5.0 MB/s  eta 0:00:01
All packages installed


In [6]:
#Download the data
url="https://en.wikipedia.org/wiki/List_of_districts_of_Istanbul"
dfs =pd.read_html(url)
df=dfs[0]
df = df.drop(df.index[[39, 40, 41, 42]], axis=0)
df.rename(columns = {"District":"District","Population (2020)":"pop2020","Area (km²)":"area","Density (per km²)":"density","Mensual household income TL(USD)":"monthly_income","Annual household income TL(USD)":"annual_income"}, inplace = True)

print(df)
df.info()

         District  pop2020     area  density    monthly_income  \
0          Adalar    16033    11.05     1451     6.652₺ (918$)   
1      Arnavutköy   296709   450.35      659     2.030₺ (279$)   
2        Ataşehir   422594    25.23    16750     6.577₺ (904$)   
3         Avcılar   436897    42.01    10400     3.662₺ (503$)   
4        Bağcılar   737206    22.36    32970     3.197₺ (441$)   
5    Bahçelievler   592371    16.62    35642     4.674₺ (645$)   
6        Bakırköy   226229    29.64     7633   8.845₺ (1,220$)   
7      Başakşehir   469924   104.30     4506     4.513₺ (622$)   
8      Bayrampaşa   269950     9.61    28091     3.480₺ (480$)   
9        Beşiktaş   176513    18.01     9801  10.560₺ (1,457$)   
10         Beykoz   246110   310.36      793     3.693₺ (509$)   
11     Beylikdüzü   365572    37.78     9676     4.327₺ (597$)   
12        Beyoğlu   226396     8.91    25409     4.773₺ (658$)   
13   Büyükçekmece   257362   139.17     1849     3.671₺ (506$)   
14        

In [16]:
# add geographical coordinate as separate columns
app = Nominatim(user_agent="myapplication")
df["latitude"] = df["District"].apply(app.geocode)
df["latitude"] = df["latitude"].apply(lambda x: (x.latitude))
df["longitude"] = df["District"].apply(app.geocode)
df["longitude"] = df["longitude"].apply(lambda x: (x.longitude))

print(df.head()) # csv = df.to_csv("districts.csv", index= True) export as csv to check data if necessary
x = len(df)
print(df.shape)
print(df.dtypes)
# add map with folium
map = folium.Map(location=[41.01384,28.94966], default_zoom_start=10)
for i in range(0,len(df)):
    folium.Marker([df.iloc[i]["latitude"], df.iloc[i]["longitude"]], popup=df.iloc[i]["District"]).add_to(map)
map.save("istanbul.html")
map

     District  pop2020    area  density monthly_income      annual_income  \
0      Adalar    16033   11.05     1451  6.652₺ (918$)  79.821₺ (10,978$)   
1  Arnavutköy   296709  450.35      659  2.030₺ (279$)   24.360₺ (3,350$)   
2    Ataşehir   422594   25.23    16750  6.577₺ (904$)  78.924₺ (10,854$)   
3     Avcılar   436897   42.01    10400  3.662₺ (503$)   43.938₺ (6,064$)   
4    Bağcılar   737206   22.36    32970  3.197₺ (441$)   38.367₺ (5,295$)   

    latitude  longitude  
0  40.876259  29.091027  
1  41.184182  28.740729  
2  40.984749  29.106720  
3  40.980135  28.717547  
4  41.033899  28.857898  
(39, 8)
District           object
pop2020             int64
area               object
density             int64
monthly_income     object
annual_income      object
latitude          float64
longitude         float64
dtype: object


In [8]:
#setting things for foursquare

results = [] # empty var
CLIENT_ID = 'EQFT4YHJOKL3XKWMOVM5NURKN0UR1JUSNFZW2Z0GC54RVS2T' # Foursquare ID
CLIENT_SECRET = 'GIJPYCBUAVNOV4AKUS0SYEY2SIZSUEKRJR15Q0VUE135TW2U' # Foursquare Secret
VERSION = '20180604'
CATEGORY = "4bf58dd8d48988d1e1931735"# category for gamearcade
RADIUS = 500
LIMIT = 100
# coordinates = [41.01384, 28.94966]
# recall data from foursquare create API request
for i in range(0, len(df)):
    name = df.iloc[i]["District"]
    lat = df.iloc [ i ] [ "latitude" ]
    lng = df.iloc [ i ] [ "longitude" ]
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            RADIUS,
            LIMIT,
            CATEGORY)
    result = requests.get(url).json()["response"]['groups'][0]['items']
    results.append ( [ (
        name,
        lat,
        lng,
        v [ 'venue' ] [ 'name' ],
        v [ 'venue' ] [ 'location' ] [ 'lat' ],
        v [ 'venue' ] [ 'location' ] [ 'lng' ],
        v [ 'venue' ] [ 'categories' ] [ 0 ] [ 'name' ]) for v in result ] )
arcade = pd.DataFrame([item for x in results for item in x])
print(arcade)
Arcade_locations = arcade.to_csv("Arcade_locations.csv", index= False)

with open("results.json","w") as json_file:
     json.dump(results, json_file)

Istanbul_venues = pd.read_csv("Arcade_locations.csv")

Istanbul_venues.columns = ["District","latitude_d","longitude_d","Venue","latitude_v","longitude_v","type"]
print(Istanbul_venues)

               0          1          2                     3          4  \
0     Arnavutköy  41.184182  28.740729  YILDIRIM PLAYSTATION  41.182449   
1     Arnavutköy  41.184182  28.740729    Yeşilçam Kahve Evi  41.182053   
2     Arnavutköy  41.184182  28.740729             Pes  Clup  41.183780   
3     Arnavutköy  41.184182  28.740729  PES CLUB PLAYSTATION  41.183716   
4     Arnavutköy  41.184182  28.740729      Room1408korkuevi  41.183785   
..           ...        ...        ...                   ...        ...   
183  Zeytinburnu  40.988118  28.903635    Simge Düğün Salonu  40.988406   
184  Zeytinburnu  40.988118  28.903635              Pes Cafe  40.991848   
185  Zeytinburnu  40.988118  28.903635  Underground İnternet  40.991621   
186  Zeytinburnu  40.988118  28.903635     Yirmibeşoğlu Cafe  40.992180   
187  Zeytinburnu  40.988118  28.903635         Shaky Leather  40.985603   

             5       6  
0    28.743928  Arcade  
1    28.744583    Café  
2    28.739187  Arcade  

K-clustering

In [9]:
# barchart = df.plot.bar(x = "District", y=type)
Istanbul_venues.groupby("District").count()
Istanbul_onehot = pd.get_dummies(Istanbul_venues[["type"]], prefix="", prefix_sep="")
Istanbul_onehot.insert(loc=0, column="District", value=Istanbul_venues["District"] )
Istanbul_onehot.shape
print(Istanbul_onehot.shape)

(188, 6)


In [10]:
Istanbul_grouped = Istanbul_onehot.groupby ( "District" ).mean ().reset_index ()
print(Istanbul_grouped)

         District    Arcade       Bar      Café  General Entertainment  \
0      Arnavutköy  0.888889  0.000000  0.111111                    0.0   
1         Avcılar  1.000000  0.000000  0.000000                    0.0   
2        Bakırköy  1.000000  0.000000  0.000000                    0.0   
3      Bayrampaşa  0.800000  0.000000  0.000000                    0.2   
4        Bağcılar  1.000000  0.000000  0.000000                    0.0   
5      Başakşehir  1.000000  0.000000  0.000000                    0.0   
6      Beylikdüzü  1.000000  0.000000  0.000000                    0.0   
7         Beyoğlu  0.666667  0.000000  0.000000                    0.0   
8        Beşiktaş  0.954545  0.045455  0.000000                    0.0   
9    Büyükçekmece  1.000000  0.000000  0.000000                    0.0   
10        Esenler  1.000000  0.000000  0.000000                    0.0   
11       Esenyurt  1.000000  0.000000  0.000000                    0.0   
12          Fatih  1.000000  0.000000 

In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc [ 1: ]
    row_categories_sorted = row_categories.sort_values ( ascending=False )

    return row_categories_sorted.index.values [ 0:num_top_venues ]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
District_venues_sorted = pd.DataFrame(columns=columns)
District_venues_sorted['District'] = Istanbul_grouped['District']

for ind in np.arange(Istanbul_grouped.shape[0]):
    District_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Istanbul_grouped.iloc[ind, :], num_top_venues)

print(District_venues_sorted)

         District 1st Most Common Venue  2nd Most Common Venue  \
0      Arnavutköy                Arcade                   Café   
1         Avcılar                Arcade               Tea Room   
2        Bakırköy                Arcade               Tea Room   
3      Bayrampaşa                Arcade  General Entertainment   
4        Bağcılar                Arcade               Tea Room   
5      Başakşehir                Arcade               Tea Room   
6      Beylikdüzü                Arcade               Tea Room   
7         Beyoğlu                Arcade               Tea Room   
8        Beşiktaş                Arcade                    Bar   
9    Büyükçekmece                Arcade               Tea Room   
10        Esenler                Arcade               Tea Room   
11       Esenyurt                Arcade               Tea Room   
12          Fatih                Arcade               Tea Room   
13  Gaziosmanpaşa                Arcade               Tea Room   
14       G

In [12]:
# set number of clusters
kclusters = 5

Istanbul_grouped_clustering = Istanbul_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Istanbul_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])

[3 0 0 2 0 0 0 1 4 0]


In [14]:
# set color scheme for the clusters
map = folium.Map(location=[41.01384,28.94966], default_zoom_start=10)
marker_cluster = MarkerCluster().add_to(map)
for i in range ( Istanbul_venues.shape [ 0 ] ):
    location = [Istanbul_venues [ 'latitude_v' ] [ i ], Istanbul_venues [ 'longitude_v' ] [ i ] ]
    tooltip = "District:{}<br> Venue: {}<br> Click for more".format ( Istanbul_venues [ "District" ] [ i ],
                                                                       Istanbul_venues [ 'Venue' ] [ i ] )

    folium.Marker ( location,  # adding more details to the popup screen using HTML
                    popup="""
                  <i>Category: </i> <br> <b>${}</b> <br>
                  <i>District: </i><b><br>{}</b><br>""".format (
                         Istanbul_venues [ 'type' ] [ i ],
                         Istanbul_venues [ 'District' ] [ i ] ),
                    tooltip=tooltip ).add_to ( marker_cluster )
map.save("Mappa.html")
map